In [1]:
import numpy as np
import pandas as pd

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
K.set_image_dim_ordering('th')


# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

Using TensorFlow backend.


In [2]:
train = pd.read_csv('kaggle_train.csv')
print(train.shape)
test = pd.read_csv('kaggle_test.csv')
print(test.shape)

(42000, 785)
(28000, 784)


In [3]:
labels = train.iloc[:, 0].values.astype('int32')
X_train = (train.iloc[:, 1:].values).astype('float32')
X_test = (test.values).astype('float32')

In [4]:
X_train = X_train.reshape(X_train.shape[0], 1, 28, 28)
X_test = X_test.reshape(X_test.shape[0], 1, 28, 28)

# one hot encode outputs
y_train = np_utils.to_categorical(labels)
num_classes = y_train.shape[1]

In [5]:
# normalize inputs from 0-255 to 0-1
X_train = X_train / 255
X_test = X_test / 255

mean = np.std(X_train)
X_train -= mean
X_test -= mean

In [6]:
model = Sequential()
model.add(Conv2D(30, (5,5), input_shape=(1, 28, 28), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(15, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

In [7]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [8]:
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.1, verbose=2)

Train on 37800 samples, validate on 4200 samples
Epoch 1/20
63s - loss: 0.2354 - acc: 0.9243 - val_loss: 0.0592 - val_acc: 0.9798
Epoch 2/20
62s - loss: 0.0766 - acc: 0.9765 - val_loss: 0.0484 - val_acc: 0.9855
Epoch 3/20
62s - loss: 0.0557 - acc: 0.9821 - val_loss: 0.0415 - val_acc: 0.9852
Epoch 4/20
62s - loss: 0.0470 - acc: 0.9852 - val_loss: 0.0381 - val_acc: 0.9862
Epoch 5/20
61s - loss: 0.0409 - acc: 0.9868 - val_loss: 0.0327 - val_acc: 0.9898
Epoch 6/20
62s - loss: 0.0339 - acc: 0.9890 - val_loss: 0.0355 - val_acc: 0.9888
Epoch 7/20
62s - loss: 0.0299 - acc: 0.9907 - val_loss: 0.0331 - val_acc: 0.9879
Epoch 8/20
61s - loss: 0.0292 - acc: 0.9909 - val_loss: 0.0334 - val_acc: 0.9895
Epoch 9/20
61s - loss: 0.0258 - acc: 0.9917 - val_loss: 0.0266 - val_acc: 0.9895
Epoch 10/20
63s - loss: 0.0232 - acc: 0.9927 - val_loss: 0.0424 - val_acc: 0.9843
Epoch 11/20
62s - loss: 0.0224 - acc: 0.9929 - val_loss: 0.0316 - val_acc: 0.9905
Epoch 12/20
62s - loss: 0.0196 - acc: 0.9939 - val_loss: 0

In [9]:
scores = model.evaluate(X_train, y_train, verbose=0)
print("Large CNN Error: %.2f%%" % (100-scores[1]*100))

Large CNN Error: 0.21%


In [10]:
print("Generating test predictions...")
preds = model.predict_classes(X_test, verbose=0)

Generating test predictions...


In [11]:
preds

array([2, 0, 9, ..., 3, 9, 2], dtype=int64)

In [12]:
def write_preds(preds, fname):
    pd.DataFrame({"ImageId": list(range(1, len(preds)+1)), "Label": preds}).to_csv(fname, index=False, header=True)
write_preds(preds, 'keras-larger_cnn-jb2.csv')  

In [13]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 30, 24, 24)        780       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 12, 12)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 15, 10, 10)        4065      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 15, 5, 5)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 15, 5, 5)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 375)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               48128     
__________